# Testing

In this notebook you can explore and test the resulting database tables from our Dagster ETL process.
Here you can analyze


In [4]:
import duckdb
import polars as pl

# from IPython import display
import geopandas as gpd
import plotly.express as px
from shapely import wkt

In [ ]:
def convert_to_geodf(polars_df: pl.DataFrame) -> gpd.GeoDataFrame:
    """
    Convert a Polars DataFrame to a GeoDataFrame using WKB or WKT transformation.
    """

    # Convert Polars DataFrame to Pandas DataFrame
    df = polars_df.to_pandas()

    # Convert geometry strings back to geometry objects
    if "geometry" in df.columns:
        df["geometry"] = df["geometry"].apply(wkt.loads)

    else:
        raise ValueError("No 'geometry' column found in the DataFrame")

    # Convert back to GeoDataFrame
    return gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")


def convert_to_polars(gdf: gpd.GeoDataFrame) -> pl.DataFrame:
    """
    Convert a GeoDataFrame to a Polars DataFrame, converting geometries to WKB strings.
    """
    # If geometry conversion is necessary, uncomment the following line
    # gdf["geometry"] = gdf["geometry"].apply(lambda geom: wkb_dumps(geom, hex=True))
    gdf["geometry"] = gdf["geometry"].apply(wkt.dumps)

    # Convert to Polars DataFrame
    return pl.from_pandas(gdf)

### Create Database Connection


In [ ]:
%load_ext sql
conn = duckdb.connect(database="../dsp-dagster/DSP.duckdb")
%sql conn --alias duckdb
%sql SHOW ALL TABLES; # shows all available tables

##### Drop Tables


In [ ]:
# conn = duckdb.connect(database="../dsp-dagster/data_systems_project.duckdb")
# %sql conn --alias duckdb
# %sql DROP TABLE joined.incident_deployments_vehicles_weather;
# %sql DROP TABLE joined.incident_deployments_vehicles_wijken;
# %sql DROP TABLE joined.incident_deployments_vehicles;
# %sql DROP TABLE joined.incidents_buurten;
# %sql DROP TABLE public.storm_incidents;
# %sql DROP TABLE joined.buurten_trees;
# %sql DROP TABLE joined.incident_tree;
# %sql DROP TABLE joined.tree_service_areas;
# conn.close()

In [ ]:
# STN         LON(east)   LAT(north)  ALT(m)      NAME
# 240         4.790       52.318      -3.30       Schiphol
# YYYYMMDD  : datum (YYYY=jaar;MM=maand;DD=dag)
# HH        : tijd (HH=uur; UT.12 UT=13 MET; 14 MEZT. Uurvak 05 loopt van 04.00 UT tot 5.00 UT
# DD        : Windrichting (in graden) gemiddeld over de laatste 10 minuten van het afgelopen uur (360=noord; 90=oost; 180=zuid; 270=west; 0=windstil 990=veranderlijk. Zie http://www.knmi.nl/kennis-en-datacentrum/achtergrond/klimatologische-brochures-en-boeken / Mean wind direction (in degrees) during the 10-minute period preceding the time of observation (360=north; 90=east; 180=south; 270=west; 0=calm 990=variable)
# FH        : Uurgemiddelde windsnelheid (in 0.1 m/s). Zie http://www.knmi.nl/kennis-en-datacentrum/achtergrond/klimatologische-brochures-en-boeken / Hourly mean wind speed (in 0.1 m/s)
# FF        : Windsnelheid (in 0.1 m/s) gemiddeld over de laatste 10 minuten van het afgelopen uur / Mean wind speed (in 0.1 m/s) during the 10-minute period preceding the time of observation
# FX        : Hoogste windstoot (in 0.1 m/s) over het afgelopen uurvak / Maximum wind gust (in 0.1 m/s) during the hourly division
# T         : Temperatuur (in 0.1 graden Celsius) op 1.50 m hoogte tijdens de waarneming / Temperature (in 0.1 degrees Celsius) at 1.50 m at the time of observation
# T10N      : Minimumtemperatuur (in 0.1 graden Celsius) op 10 cm hoogte in de afgelopen 6 uur / Minimum temperature (in 0.1 degrees Celsius) at 0.1 m in the preceding 6-hour period
# TD        : Dauwpuntstemperatuur (in 0.1 graden Celsius) op 1.50 m hoogte tijdens de waarneming / Dew point temperature (in 0.1 degrees Celsius) at 1.50 m at the time of observation
# SQ        : Duur van de zonneschijn (in 0.1 uren) per uurvak; berekend uit globale straling  (-1 for <0.05 uur) / Sunshine duration (in 0.1 hour) during the hourly division; calculated from global radiation (-1 for <0.05 hour)
# Q         : Globale straling (in J/cm2) per uurvak / Global radiation (in J/cm2) during the hourly division
# DR        : Duur van de neerslag (in 0.1 uur) per uurvak / Precipitation duration (in 0.1 hour) during the hourly division
# RH        : Uursom van de neerslag (in 0.1 mm) (-1 voor <0.05 mm) / Hourly precipitation amount (in 0.1 mm) (-1 for <0.05 mm)
# P         : Luchtdruk (in 0.1 hPa) herleid naar zeeniveau; tijdens de waarneming / Air pressure (in 0.1 hPa) reduced to mean sea level; at the time of observation
# VV        : Horizontaal zicht tijdens de waarneming (0=minder dan 100m; 1=100-200m; 2=200-300m;...; 49=4900-5000m; 50=5-6km; 56=6-7km; 57=7-8km; ...; 79=29-30km; 80=30-35km; 81=35-40km;...; 89=meer dan 70km) / Horizontal visibility at the time of observation (0=less than 100m; 1=100-200m; 2=200-300m;...; 49=4900-5000m; 50=5-6km; 56=6-7km; 57=7-8km; ...; 79=29-30km; 80=30-35km; 81=35-40km;...; 89=more than 70km)
# N         : Bewolking (bedekkingsgraad van de bovenlucht in achtsten); tijdens de waarneming (9=bovenlucht onzichtbaar) / Cloud cover (in octants); at the time of observation (9=sky invisible)
# U         : Relatieve vochtigheid (in procenten) op 1.50 m hoogte tijdens de waarneming / Relative atmospheric humidity (in percents) at 1.50 m at the time of observation
# WW        : Weercode (00-99); visueel(WW) of automatisch(WaWa) waargenomen; voor het actuele weer of het weer in het afgelopen uur. Zie http://bibliotheek.knmi.nl/scholierenpdf/weercodes_Nederland / Present weather code (00-99); description for the hourly division.
# IX        : Weercode indicator voor de wijze van waarnemen op een bemand of automatisch station (1=bemand gebruikmakend van code uit visuele waarnemingen; 2;3=bemand en weggelaten (geen belangrijk weersverschijnsel; geen gegevens); 4=automatisch en opgenomen (gebruikmakend van code uit visuele waarnemingen); 5;6=automatisch en weggelaten (geen belangrijk weersverschijnsel; geen gegevens); 7=automatisch gebruikmakend van code uit automatische waarnemingen) / Indicator present weather code (1=manned and recorded (using code from visual observations); 2;3=manned and omitted (no significant weather phenomenon to report; not available); 4=automatically recorded (using code from visual observations); 5;6=automatically omitted (no significant weather phenomenon to report; not available); 7=automatically set (using code from automated observations)
# M         : Mist 0=niet voorgekomen; 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming / Fog 0=no occurrence; 1=occurred during the preceding hour and/or at the time of observation
# R         : Regen 0=niet voorgekomen; 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming / Rainfall 0=no occurrence; 1=occurred during the preceding hour and/or at the time of observation
# S         : Sneeuw 0=niet voorgekomen; 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming / Snow 0=no occurrence; 1=occurred during the preceding hour and/or at the time of observation
# O         : Onweer 0=niet voorgekomen; 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming / Thunder  0=no occurrence; 1=occurred during the preceding hour and/or at the time of observation
# Y         : IJsvorming 0=niet voorgekomen; 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming / Ice formation 0=no occurrence; 1=occurred during the preceding hour and/or at the time of observation

In [ ]:
conn = duckdb.connect(database="../dsp-dagster/DSP.duckdb")
%sql conn --alias duckdb
# weather_incident = conn.execute(
#     """
#     SELECT * FROM joined.weather_incident
#     """
# ).pl()
# display(weather_incident.head())

# tree_service_areas = conn.execute(
#     """
#     SELECT * FROM joined .tree_service_areas
#     """
# ).pl()
# display(tree_service_areas.head())

incident_tree = conn.execute(
    """
    SELECT * FROM joined.incident_tree
    """
).pl()
display(incident_tree.head(5))
print(incident_tree.shape)
print(incident_tree.null_count())

new_storm_incidents = conn.execute(
    """
    SELECT * FROM clean.new_storm_incidents
    """
).pl()
display(new_storm_incidents.head(5))
print(new_storm_incidents.shape)

cbs_buurten = conn.execute(
    """
    SELECT * FROM public.cbs_buurten
    """
).pl()
display(cbs_buurten.head(5))
print(cbs_buurten.shape)

# knmi_weather_txt = conn.execute(
#     """
#     SELECT * FROM public.knmi_weather_txt
#     """
# ).pl()
# display(weather_incident.head())

# Close the database connection
conn.close()

In [ ]:
cbs_buurten_selection = cbs_buurten.select("buurtcode", "buurtnaam", "geometry")
cbs_buurten_selection.head()

In [ ]:
# DD        : Windrichting (in graden) gemiddeld over de laatste 10 minuten van het afgelopen uur (360=noord; 90=oost; 180=zuid; 270=west; 0=windstil 990=veranderlijk. Zie http://www.knmi.nl/kennis-en-datacentrum/achtergrond/klimatologische-brochures-en-boeken / Mean wind direction (in degrees) during the 10-minute period preceding the time of observation (360=north; 90=east; 180=south; 270=west; 0=calm 990=variable)
# FH        : Uurgemiddelde windsnelheid (in 0.1 m/s). Zie http://www.knmi.nl/kennis-en-datacentrum/achtergrond/klimatologische-brochures-en-boeken / Hourly mean wind speed (in 0.1 m/s)
# FF        : Windsnelheid (in 0.1 m/s) gemiddeld over de laatste 10 minuten van het afgelopen uur / Mean wind speed (in 0.1 m/s) during the 10-minute period preceding the time of observation
# FX        : Hoogste windstoot (in 0.1 m/s) over het afgelopen uurvak / Maximum wind gust (in 0.1 m/s) during the hourly division
# T         : Temperatuur (in 0.1 graden Celsius) op 1.50 m hoogte tijdens de waarneming / Temperature (in 0.1 degrees Celsius) at 1.50 m at the time of observation
# WW        : Weercode (00-99); visueel(WW) of automatisch(WaWa) waargenomen; voor het actuele weer of het weer in het afgelopen uur. Zie http://bibliotheek.knmi.nl/scholierenpdf/weercodes_Nederland / Present weather code (00-99); description for the hourly division.
# R         : Regen 0=niet voorgekomen; 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming / Rainfall 0=no occurrence; 1=occurred during the preceding hour and/or at the time of observation
# S         : Sneeuw 0=niet voorgekomen; 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming / Snow 0=no occurrence; 1=occurred during the preceding hour and/or at the time of observation
# O         : Onweer 0=niet voorgekomen; 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming / Thunder  0=no occurrence; 1=occurred during the preceding hour and/or at the time of observation

In [ ]:
incident_tree_selection = incident_tree.drop("Incident_Duration", "Incident_Endtime", "Incident_Priority", "Incident_Starttime", "typeSoortnaam", "typeEigenaarPlus", "jaarVanAanleg", "soortnaamKort", "soortnaamTop", "Municipality")
display(incident_tree_selection.head(5))
incident_tree_selection.shape
pl.Config.set_fmt_str_lengths(100)





In [ ]:
gdf1 = convert_to_geodf(incident_tree_selection)
# Convert the geometry to latitude and longitude
gdf1['latitude'] = gdf1.geometry.y
gdf1['longitude'] = gdf1.geometry.x

# Plotting using Plotly Express
fig = px.scatter_mapbox(gdf1,
                        lat='latitude',
                        lon='longitude',
                        zoom=10)

# Example GeoDataFrame for the second plot
gdf2 = convert_to_geodf(new_storm_incidents)
gdf2['latitude'] = gdf2.geometry.y
gdf2['longitude'] = gdf2.geometry.x

# Add the second plot over the first one
fig.add_trace(px.scatter_mapbox(gdf2,
                                lat='latitude',
                                lon='longitude',  # Replace with your second column name
                               ).data[0])


# Update the layout
fig.update_layout(mapbox_style="open-street-map")
fig.show()

In [ ]:
gdf_buurten = convert_to_geodf(cbs_buurten_selection)
gdf_incident_tree = convert_to_geodf(incident_tree_selection)

# df.group_by(["Incident_Date", "IncStrt_Hour","Service_Area", "soortnaam", "boomhoogteklasseActueel", "leeftijdBoom", "typeObject", "Damage_Type"])

# selection = weather_incident.select(["DD", "FH", "FF", "FX", "T", "WW", "R", "S", "O", "Incident_ID", "Damage_Type", "Service_Area", "geometry"])
# groupby_selection = selection.group_by(by=["DD", "FH", "FF", "FX", "T", "WW", "R", "S", "O", "Service_Area","Damage_Type"]).agg(Incident_Counts=pl.col("Incident_ID").count())
# # groupby_selection.filter(pl.col("Incident_Counts") > 0)


# df = groupby_selection.join(tree_service_areas, how="left", left_on="Service_Area", right_on="Verzorgingsgebied")
# df.head()

In [ ]:
# df = new_storm_incidents.group_by("Damage_Type").agg(pl.col("Incident_ID").count().alias("Total"))
# px.histogram(df.to_pandas(), x="Damage_Type", y="Total")

In [ ]:
for service_area in tree_service_areas["Verzorgingsgebied"].unique():
    df = tree_service_areas.filter(pl.col("Verzorgingsgebied") == service_area)
    print(f"{service_area} df has shape: {df.shape}")
    df = tree_service_areas.join()
    break